In [2]:
import urllib.request

# coco.names
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names",
    "coco.names"
)

# yolov3.cfg
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg",
    "yolov3.cfg"
)

# yolov3.weights (this is ~248MB, so it may take time)
urllib.request.urlretrieve(
    "https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov3.weights",
    "yolov3.weights"
)

# Sample video for object detection
urllib.request.urlretrieve(
    "https://github.com/opencv/opencv/raw/master/samples/data/vtest.avi",
    "test_video.avi"
)


('test_video.avi', <http.client.HTTPMessage at 0x200aee87ad0>)

In [3]:
import cv2
import numpy as np
import time
import os

In [4]:
# Load class labels
with open('coco.names', 'r') as f:
    LABELS = f.read().strip().split('\n')

In [5]:
# Generate unique colors for each label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype='uint8')

In [6]:
# Load YOLOv3 weights and config
print("[INFO] Loading YOLOv3 model...")
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

[INFO] Loading YOLOv3 model...


In [7]:
# Load YOLOv3 weights and config
print("[INFO] Loading YOLOv3 model...")
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

[INFO] Loading YOLOv3 model...


In [13]:
# Get output layer names (works across OpenCV versions)
try:
    ln = net.getUnconnectedOutLayersNames()  # OpenCV 4.2.0 and above
except AttributeError:
    # For older OpenCV versions
    ln = net.getLayerNames()
    ln = [ln[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Initialize webcam (0 = default webcam)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("[ERROR] Cannot access webcam")
    exit()

print("[INFO] Starting real-time detection. Press 'q' to quit...")

while True:
    ret, frame = cap.read()
    if not ret:
        print("[WARNING] Frame not received. Exiting...")
        break

    (H, W) = frame.shape[:2]

    # Create a blob and perform forward pass
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    boxes = []
    confidences = []
    classIDs = []

    # Set thresholds
    confidence_threshold = 0.5
    nms_threshold = 0.3

    # Process detections
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > confidence_threshold:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # Apply Non-Max Suppression
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

    # Draw boxes
    if len(idxs) > 0:
        for i in idxs.flatten():
            x, y, w, h = boxes[i]
            color = [int(c) for c in COLORS[classIDs[i]]]
            label = f"{LABELS[classIDs[i]]}: {confidences[i]:.2f}"

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display output
    cv2.imshow("YOLOv3 Real-Time Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
print("[INFO] Detection stopped. Resources released.")

[INFO] Starting real-time detection. Press 'q' to quit...
[INFO] Detection stopped. Resources released.


In [ ]:
# Cleanup
cap.release()
cv2.destroyAllWindows()
print("[INFO] Detection stopped. Resources released.")